In [1]:
from numpy import pi, linspace, diff, mean, min, max, arctan2, dot
from skyfield.api import load
from skyfield.framelib import ecliptic_J2000_frame
from skyfield.searchlib import find_discrete
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [2]:
planet_name = "mercury"
ref_name = "sun"
ts = load.timescale()
eph_s_name = 'de421.bsp'
# eph_s_name = 'de422.bsp'
t_start = ts.tt(1900, 1, 1, 0, 0, 0)
period_days = 365*150
eph_s = load(eph_s_name)
earth = eph_s["earth"]
sun = eph_s["sun"]
planet = eph_s[planet_name]
ref = eph_s[ref_name]

def get_fill_colors(times):
    return [f"#{255:02x}{int(255 * (times.tt[-1] - t) / (times.tt[-1] - times.tt[0])):02x}{255:02x}" for t in times.tt]

## Distance from Sun

In [12]:
times_plot = ts.tt_jd(linspace(t_start.tt, t_start.tt + 365, 1000))
lat, lon, dist = (planet.at(times_plot) - ref.at(times_plot)).frame_latlon(ecliptic_J2000_frame)
pos_vec, v_vec = (planet.at(times_plot) - ref.at(times_plot)).frame_xyz_and_velocity(ecliptic_J2000_frame)

mean(dist.km), min(dist.km), max(dist.km)

(np.float64(59408951.1893783),
 np.float64(46001277.4995958),
 np.float64(69816528.64108366))

In [4]:
fig = figure(title=planet_name.capitalize() + " distance", x_axis_label="days", y_axis_label="distance (au)", width=1100)
fig.scatter(times_plot.tt-t_start.tt, dist.au, line_color="red", fill_color = get_fill_colors(times_plot))
fig.line(times_plot.tt-t_start.tt, dist.au, line_color="red")
show(fig)

## The movement of Perigee

In [5]:
def dist_vel_zero(target, ref):
    def zero(t):
        pos_vec, v_vec = (planet.at(t) - ref.at(t)).frame_xyz_and_velocity(ecliptic_J2000_frame)
        v_x, v_y, v_z = v_vec.km_per_s
        x, y, z = pos_vec.km
        return v_x*x + v_y*y + v_z*z > 0
    zero.step_days = 10.0
    return zero

In [6]:
vel_zero = find_discrete(t_start, t_start + period_days, dist_vel_zero(planet, ref))
t_dist_max = vel_zero[0][vel_zero[1] == 0]
lat, lon, dist = (planet.at(t_dist_max) - ref.at(t_dist_max)).frame_latlon(ecliptic_J2000_frame)
pos_max_vel, v_max_vel = (planet.at(t_dist_max) - ref.at(t_dist_max)).frame_xyz_and_velocity(ecliptic_J2000_frame)

In [7]:
fig = figure(title=planet_name.capitalize()+" max dist location", x_axis_label="x(au)", y_axis_label="y(au)", width=1000, height=1000)
fig.scatter(pos_max_vel.au[0], pos_max_vel.au[1], fill_color=get_fill_colors(t_dist_max), line_color="red")
fig.line(pos_max_vel.au[0], pos_max_vel.au[1], line_color="red")
show(fig)

In [8]:
fig = figure(title=planet_name.capitalize()+" max distance relative latitude", x_axis_label="days", y_axis_label="arcsec", width=1100)
fig.scatter(t_dist_max.tt - t_start.tt, (lat.degrees-lat.degrees[0])*3600, fill_color=get_fill_colors(t_dist_max), line_color="red")
fig.line(t_dist_max.tt - t_start.tt, (lat.degrees-lat.degrees[0])*3600, line_color="red")
show(fig)

In [9]:
fig = figure(title=planet_name.capitalize()+" max distance relative longitude", x_axis_label="days", y_axis_label="arcsec", width=1100)
fig.scatter(t_dist_max.tt - t_start.tt, (lon.degrees-lon.degrees[0])*3600, fill_color=get_fill_colors(t_dist_max), line_color="blue")
fig.line(t_dist_max.tt - t_start.tt, (lon.degrees-lon.degrees[0])*3600, line_color="blue")
show(fig)

Mercury aphelion (perihelion) precession rate in arcseconds/century, in close agreement with official rate of 574:

In [10]:
(lon.degrees[-1]-lon.degrees[0])/(t_dist_max.tt[-1] - t_start.tt)*365.25*3600*100

np.float64(572.3103491907566)